# MCMC with Planck Likelihoods

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import py21cmmc as py21c

from py21cmmc import analyse
from py21cmmc import mcmc
from py21cmmc import likelihood
from py21cmmc import core


%load_ext autoreload
%autoreload 2

import logging

In this tutorial we demonstrate how to do MCMC with a lightcone, to fit just two astrophysical parameters without noise, and then visualise the results. This tutorial follows a very similar pattern to the MCMC intro, and you should follow that one first.

## Running MCMC

To perform an MCMC on a lightcone is *very* similar to a Coeval cube. Merely use the ``CoreLightConeModule`` as the core module, and the ``Likelihood1DPowerLightcone`` as the likelihood. One extra parameter to the ``core`` is available -- ``max_redshift``, which specifies the approximate upper limit on the lightcone's depth. Note that this does **not** necessarily specify the maximum redshift at which the ionization will be computed (this is specified by ``z_heat_max``), it merely specifies where to start saving the ionization boxes into a lightcone. 

Furthermore, one extra parameter to the likelihood is available -- ``nchunks`` -- which allows to break the full lightcone up into independent chunks for which the power spectrum will be computed.

Thus, for example:

In [8]:
core = py21c.CoreLightConeModule( # All core modules are prefixed by Core* and end with *Module
    redshift = 6.0,              # Lower redshift of the lightcone
    max_redshift = 20.0,          # Approximate maximum redshift of the lightcone (will be exceeded).
    user_params = dict(       
        HII_DIM = 50,         
        BOX_LEN = 125.0
    ),
    z_step_factor=1.04,          # How large the steps between evaluated redshifts are (log).
    z_heat_max=18.0,             # Completely ineffective since no spin temp or inhomogeneous recombinations.
    regenerate=False          
) # For other available options, see the docstring.
coreCMB = py21c.CoreCMB(verbose = 1)
# Now the likelihood...
#datafiles = ["data/lightcone_mcmc_data_%s.npz"%i for i in range(4)]
#likelihood = mcmc.Likelihood1DPowerLightcone(  # All likelihood modules are prefixed by Likelihood*
#    datafile = datafiles,        # All likelihoods have this, which specifies where to write/read data
#    logk=False,                 # Should the power spectrum bins be log-spaced?
#    min_k=0.1,                  # Minimum k to use for likelihood
#    max_k=1.0,                  # Maximum ""
#    nchunks = 4,                 # Number of chunks to break the lightcone into
#    simulate=True
#) # For other available options, see the docstring


import CLASS


In [9]:
planckEE = "/Users/poulin/Dropbox/Labo/baseline/plc_3.0/low_l/simall/simall_100x143_offlike5_EE_Aplanck_B.clik/"
#likelihood.initialize_clik_and_class(planckEE)

In [10]:
likelihoodPSEE = py21c.LikelihoodPlanckPowerSpectra(  # All likelihood modules are prefixed by Likelihood*
    datafolder = planckEE,        # All likelihoods have this, which specifies where to write/read data
    name_lkl = 'Planck_lowl_EE') # For other available options, see the docstring

import clik


In [11]:
Planck_lensing = "/Users/poulin/Dropbox/Labo/baseline/plc_3.0/lensing/smicadx12_Dec5_ftl_mv2_ndclpp_p_teb_consext8.clik_lensing"
likelihoodPSLensing = py21c.LikelihoodPlanckPowerSpectra(  # All likelihood modules are prefixed by Likelihood*
    datafolder = Planck_lensing,        # All likelihoods have this, which specifies where to write/read data
    name_lkl = 'Planck_lensing') # For other available options, see the docstring

import clik


In [12]:
planckTTTEEE = "/Users/poulin/Dropbox/Labo/baseline/plc_3.0/hi_l/plik_lite/plik_lite_v22_TT.clik" 
likelihoodPSTTTEEE = py21c.LikelihoodPlanckPowerSpectra(  # All likelihood modules are prefixed by Likelihood*
    datafolder = planckTTTEEE,        # All likelihoods have this, which specifies where to write/read data
    name_lkl = 'Planck_highl_TTTEEE') # For other available options, see the docstring

Likelihoodtau = py21c.LikelihoodPlanck()

import clik


AttributeError: The path to the .clik file for the likelihood Planck_highl_TTTEEE was not found where indicated:
/Users/poulin/Dropbox/Labo/baseline/plc_3.0/hi_l/plik_lite/plik_lite_v22_TT.clik
 Note that the default path to search for it is one directory above the path['clik'] field. You can change this behaviour in all the Planck_something.data, to reflect your local configuration, or alternatively, move your .clik files to this place.

In [19]:
chains = mcmc.build_computation_chain([core,coreCMB],likelihoodPSLensing)

In [20]:
ctx = chains.build_model_data()

In [21]:
chains.computeLikelihoods(ctx)

-4.470836351496132

In [ ]:
model_name = "LightconeTest_Planck"

chain = mcmc.run_mcmc(
    [core,coreCMB], [likelihoodPSTTTEEE,likelihoodPSEE,likelihoodPSLensing],        # Use lists if multiple cores/likelihoods required. These will be eval'd in order.
#         [core], [Likelihoodtau],        # Use lists if multiple cores/likelihoods required. These will be eval'd in order.
    datadir='data',          # Directory for all outputs
    model_name=model_name,   # Filename of main chain output
    params=dict(             # Parameter dict as described above.
        HII_EFF_FACTOR = [30.0, 10.0, 50.0, 3.0],
        ION_Tvir_MIN = [4.7, 4, 6, 0.1],
    ), 
    walkersRatio=2,         # The number of walkers will be walkersRatio*nparams
    burninIterations=0,      # Number of iterations to save as burnin. Recommended to leave as zero.
    sampleIterations=100,    # Number of iterations to sample, per walker.
    threadCount=2,           # Number of processes to use in MCMC (best as a factor of walkersRatio)
    continue_sampling=False,  # Whether to contine sampling from previous run *up to* sampleIterations.
    log_level_stream=logging.DEBUG
)


2019-11-02 13:35:36,347 INFO:Using CosmoHammer 0.6.1
2019-11-02 13:35:36,348 INFO:Using emcee 2.2.1
2019-11-02 13:35:36,360 INFO:all burnin iterations already completed
2019-11-02 13:35:36,362 INFO:Sampler: <class 'py21cmmc.cosmoHammer.CosmoHammerSampler.CosmoHammerSampler'>
configuration: 
  Params: [30.   4.7]
  Burnin iterations: 0
  Samples iterations: 100
  Walkers ratio: 2
  Reusing burn in: True
  init pos generator: SampleBallPositionGenerator
  stop criteria: IterationStopCriteriaStrategy
  storage util: <py21cmmc.cosmoHammer.storage.HDFStorageUtil object at 0x124fd86d0>
likelihoodComputationChain: 
Core Modules: 
  CoreLightConeModule
  CoreCMB
Likelihood Modules: 
  LikelihoodPlanckPowerSpectra
  LikelihoodPlanckPowerSpectra
  LikelihoodPlanckPowerSpectra

2019-11-02 13:35:36,365 INFO:start sampling after burn in
2019-11-02 13:41:55,344 INFO:Iteration finished:10
2019-11-02 13:47:34,398 INFO:Iteration finished:20
2019-11-02 13:53:08,860 INFO:Iteration finished:30
2019-11-02 

## Analysis

### Accessing chain data

Access the samples object within the chain (see the intro for more details):

In [ ]:
samples = chain.samples

### Trace Plot

Often, for diagnostic purposes, the most useful plot to start with is the trace plot. This enables quick diagnosis of burnin time and walkers that haven't converged. The function in ``py21cmmc`` by default plots the log probability along with the various parameters that were fit. It also supports setting a starting iteration, and a thinning amount. 

In [ ]:
analyse.trace_plot(samples, include_lnl=True, start_iter=0, thin=1, colored=False, show_guess=True);

### Corner Plot

In [ ]:
analyse.corner_plot(samples);

### Model Comparison Plot

Extract all blob data from the samples:

In [ ]:
blobs = samples.get_blobs()

Read in the data:

In [ ]:
delta_data = [d['delta'] for d in likelihood.data]
k_data = [d['k'] for d in likelihood.data]

Now, let's define a function which will plot our model comparison:

In [ ]:
def model_compare_plot(samples, k_data, delta_data, thin=1, start_iter=0):
    chain = samples.get_chain(thin=thin, discard=start_iter, flat=True)
    blobs = samples.get_blobs(thin=thin, discard=start_iter, flat=True)
    
    ks = [blobs[name] for name in samples.blob_names if name.startswith("k")]
    models = [blobs[name] for name in samples.blob_names if name.startswith("delta")]
    
    fig, ax = plt.subplots(1, len(ks), sharex=True, sharey=True, figsize=(5*len(ks), 4.5), 
                          subplot_kw={"xscale":'log', "yscale":'log'}, gridspec_kw={"hspace":0.05, 'wspace':0.05},
                          squeeze=False)

    for i,(k,model, kd, data) in enumerate(zip(ks,models, k_data, delta_data)):
        label="models"

        for pp in model:
            ax[0,i].plot(k[0], pp, color='k', alpha=0.2, label=label, zorder=1)
            if label:
                label=None

        mean = np.mean(model, axis=0)
        std = np.std(model, axis=0)
        md = np.median(model, axis=0)

        ax[0,i].fill_between(k[0], mean - std, mean+std, color="C0", alpha=0.6)
        ax[0,i].plot(k[0], md, color="C0", label="median model")

        ax[0,i].errorbar(kd, data, yerr = (0.15*data), color="C1", 
                     label="data", ls="None", markersize=5, marker='o')

        ax[0,i].set_xlabel("$k$ [Mpc$^{-3}$]", fontsize=15)
        ax[0,i].text(0.5, 0.86, "Chunk %s"%i, transform=ax[0,i].transAxes, fontsize=15, fontweight='bold')

    ax[0,0].legend(fontsize=12)
    #plt.ylim((3.1e2, 3.5e3))


    ax[0,0].set_ylabel("$k^3 P$", fontsize=15)

#plt.savefig(join(direc, modelname+"_power_spectrum_plot.pdf"))

In [ ]:
model_compare_plot(samples, k_data, delta_data, thin=5)

In [ ]:
common_settings = {
           'output' : 'tCl, pCl, lCl',
           'lensing':'yes',
           'l_max_scalars':3000,
           # LambdaCDM parameters
           #'h':h,
           #'omega_b':omega_b,
           #'omega_cdm':omega_cdm,
           #'A_s':A_s,
           #'n_s':n_s,
           # Take fixed value for primordial Helium (instead of automatic BBN adjustment)
           #'reio_parametrization':'reio_camb',
           #'reio_inter_num':len(xe),
           #'reio_inter_z':','.join([str(x) for x in redshift_class]), #str(redshift_class),
           #'reio_inter_xe':','.join([str(x) for x in xe]),
            'input_verbose': 1,
            'background_verbose': 1,
            'thermodynamics_verbose': 1,
            'perturbations_verbose': 1,
            'transfer_verbose': 1,
            'primordial_verbose': 1,
            'spectra_verbose': 1,
            'nonlinear_verbose': 1,
            'lensing_verbose': 1,
            'output_verbose': 1}


##############
#
# call CLASS
#
###############
cosmo = Class()
cosmo.struct_cleanup()
#cosmo.set(common_settings)
#cosmo.set({'omega_b':0.022032,'omega_cdm':0.12038,'h':0.67556,'A_s':2.215e-9,'n_s':0.9619,'tau_reio':0.0925})
cosmo.set({'output':'tCl,mPk','lensing':'no','P_k_max_1/Mpc':3.0,'l_max_scalars':3000})
#print('starting to compute Class with:',common_settings)
cosmo.compute()
thermo = cosmo.get_thermodynamics()
#thermo.items()
xe_class = thermo['x_e']
z_class = thermo['z']
print('xe, z ',xe_class,z_class)
l_max=2000
cl=cosmo.raw_cl(int(l_max))
print(cl)
derived = cosmo.get_current_derived_parameters(['tau_rec','conformal_age'])
print(derived)


kk = np.logspace(-4,np.log10(3),1000)
Pk = []
for k in kk:
    Pk.append(cosmo.pk(k,0.)) # function .pk(k,z)
#print(Pk)
plt.figure(2)
plt.xscale('log');plt.yscale('log');plt.xlim(kk[0],kk[-1])
plt.xlabel(r'$k \,\,\,\, [h/\mathrm{Mpc}]$')
plt.ylabel(r'$P(k) \,\,\,\, [\mathrm{Mpc}/h]^3$')
plt.plot(kk,Pk,'b-')
plt.show()
    